# IVF Implementation 

### Depnedncies

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import pickle
import joblib
import vec_db
import heapq
import logging
from sklearn.metrics.pairwise import euclidean_distances
logging.basicConfig(level=logging.INFO)


### IVF Class

In [24]:
class IVF_Flat_Index:
    def __init__(self, n_clusters, file_name,random_state=4):
        self.n_clusters = n_clusters
        self.cluster_centers = None
        self.inverted_index = {}
        self.random_state=random_state
        self.file_name=file_name
    #! get the index clusters
    def fit(self, data):
        #! fit the data to the kmeans
        kmeans=KMeans(n_clusters=self.n_clusters,random_state=self.random_state,max_iter=1000,tol=1e-6,verbose=1,algorithm='lloyd')
        labels=kmeans.fit_predict(data)
        self.cluster_centers=kmeans.cluster_centers_
        #! create the inverted index
        for i,label in enumerate(labels):
            if label not in self.inverted_index:
                self.inverted_index[label]=[]
            self.inverted_index[label].append(i)
        #! convert the lists to numpy arrays
        for key in self.inverted_index:
            self.inverted_index[key]=np.array(self.inverted_index[key])
        
    #! save index to a file
    def save(self):
        with open(self.file_name, 'wb') as f:
            np.save(f, self.cluster_centers)
            pickle.dump(self.inverted_index, f)
    def load(self):
        with open(self.file_name, 'rb') as f:
            self.cluster_centers = np.load(f)
            self.inverted_index = pickle.load(f)
            self.n_clusters=self.cluster_centers.shape[0]
    #? To be edited instead of flat search
    def retrieve(self,query_vector,n_clusters,n_arrays,cosine_similarity,get_row):
        #! calculate the similarities between the query vector and the cluster centers
        similarities = np.array([cosine_similarity(query_vector, center) for center in self.cluster_centers])
        #! get the n nearest clusters
        nearest_clusters = np.argpartition(similarities, -n_clusters)[-n_clusters:]
        #! get nearest n arrays within nearest k clusters 
        vectors_indices = [self.inverted_index[cluster] for cluster in nearest_clusters]
        all_vectors_indices = np.concatenate(vectors_indices)
        vectors = np.array([get_row(i) for i in all_vectors_indices])
        similarities = np.array([cosine_similarity(query_vector, vector) for vector in vectors])
        #! get nearest n arrays overall
        nearest_arrays = np.argpartition(similarities, -n_arrays)[-n_arrays:]
        return vectors[nearest_arrays]


## Testing the Implementation

## generating index file 

In [25]:
# VecDB=vec_db.VecDB(db_size=1000000)
# index = IVF_Flat_Index(n_clusters=10,file_name=VecDB.index_path)
# data = VecDB.get_all_rows()
# index.fit(data)
# index.save()

## getting closest vectors using linear search

In [26]:
# #! generating 10 random query vectors each of lengh 70
# query_vectors = np.random.rand(5, 70)
# query_vectors=np.append(query_vectors,data[:5],axis=0)
# #! retreive closest vector to each query using linear search
# similars_linear = []
# for query_vector in query_vectors:
#     similarities=np.array([VecDB._cal_score(query_vector,i) for i in data])
#     nearest_arrays=np.argpartition(similarities,-1)[-1:]
#     similars_linear.append(data[nearest_arrays])
# similars_linear=np.array(similars_linear)
# similars_linear=similars_linear.reshape(similars_linear.shape[0],similars_linear.shape[2])
# print(similars_linear.shape)
# for i in range(query_vectors.shape[0]):
#     print(f"Query {i+1} linear search:\n {data[i]}")
#     print(f"Similar to:\n {similars_linear[i]}")
#     print("Similarity: ",VecDB._cal_score(query_vectors[i],similars_linear[i]))
    

## Getting closest vectors using the index

In [27]:
# index.load()

In [28]:
# similar_index=[]
# for query_vector in query_vectors:
#     similar_index.append(index.retrieve(query_vector,10,1,VecDB._cal_score,VecDB.get_one_row))
# similar_index=np.array(similar_index)
# similar_index=similar_index.reshape(similar_index.shape[0],similar_index.shape[2])
# for i in range(query_vectors.shape[0]):
#     print(f"Query {i+1} linear search:\n {data[i]}")
#     print(f"Similar to:\n {similar_index[i]}")
#     print("Similarity: ",VecDB._cal_score(query_vectors[i],similar_index[i]))

In [29]:
# print(np.all(similar_index == similars_linear, axis=1))

In [30]:
labels = np.array([0, 1, 0, 2, 1])  # Labels from KMeans
indices = np.arange(len(labels))    # Indices of the data points

# Create a dictionary with labels as keys and indices as values
label_indices = {label: indices[labels == label] for label in np.unique(labels)}
print(label_indices)

{0: array([0, 2]), 1: array([1, 4]), 2: array([3])}


In [31]:

class IVF_PQ_Index:
    def __init__(self, n_subvectors,n_bits, n_clusters,random_state=42,dimension=70):
        #! number of bits per subvector codeword
        self.n_bits = n_bits
        #! number of subvectors 
        self.n_subvectors=n_subvectors
        #! number of clusters per subvector
        self.n_clusters_per_subvector = 2**n_bits
        #! dimension of the vectors
        self.dimension = dimension
        #! number of IVF clusters
        self.n_clusters=n_clusters
        #! IVF cluster centres
        self.cluster_centers = None
        self.random_state=42
        self.inverted_index = {}

        if dimension % n_subvectors != 0:
            raise ValueError("dimension needs to be a multiple of n_subvectors")

        self.subvector_estimators = [
            [
                KMeans(
                    n_clusters=self.n_clusters_per_subvector,
                    init='random',
                    max_iter=50,
                    random_state=42
                ) 
                for _ in range(self.n_subvectors)
            ]
            for _ in range(self.n_clusters)
        ]
        logging.info(f"Created subvector estimators {self.subvector_estimators[0]!r}")

        self.is_trained = False

    def fit(self, vectors):
        if self.is_trained:
            raise ValueError("IVF is already trained.")
        #! fit the data to the kmeans
        kmeans=KMeans(n_clusters=self.n_clusters,random_state=self.random_state,verbose=1)
        labels=kmeans.fit_predict(vectors)
        self.cluster_centers=kmeans.cluster_centers_
        #! separate labels for subclusters training
        label_indices = {label: np.where(labels == label)[0] for label in np.unique(labels)}
        #! train the subclusters and assign the codewords and create the inverted index
        for i in range(self.n_clusters):
            indices = label_indices[i]
            self._train_subclusters(vectors[indices],self.subvector_estimators[i])
            codewords = self._add(vectors[indices],self.subvector_estimators[i])
            self.inverted_index[i] = (codewords, indices)
        self.is_trained = True
        

    def _train_subclusters(self, vectors,estimators):
        for i in range(self.n_subvectors):
            #! to slice arrays
            data_slicer=self.dimension//self.n_subvectors 
            estimator = estimators[i]
            subvectors = vectors[:, i * data_slicer : (i + 1) * data_slicer]
            logging.info(f"Fitting KMeans for the {i+1}-th subvectors")
            estimator.fit(subvectors)

    def _encode(self, vectors,estimators):
        #! result array to store the codewords
        result = np.empty((vectors.shape[0], self.n_subvectors), dtype=np.uint32)
        for i in range(self.n_subvectors):
            #! predict the assigned cluster for each group of subvectors
            estimator =estimators[i]
            #! to slice arrays
            data_slicer=self.dimension//self.n_subvectors
            query = vectors[:, i * data_slicer : (i + 1) * data_slicer]
            result[:, i] = estimator.predict(query)

        return result
    def _add(self, vectors,estimators):
        codewords = self._encode(vectors,estimators)
        codewords = codewords.astype(np.uint16)
        return codewords

    def _subvector_distance(self, queries,codewords,cluster_index):
        if not self.is_trained:
            raise ValueError("Index is not trained yet")
        #! table to store the distances between the query subvectors and the codewords subvector
        distances_table = np.zeros(( queries.shape[0],self.n_subvectors,self.n_clusters_per_subvector), dtype=np.float32)
        #! calculate the distance between the queries sub vectors and the clusters centers for each subvector
        for i in range(self.n_subvectors):
            #! to slice arrays
            data_slicer=self.dimension//self.n_subvectors
            query = queries[:, i * data_slicer : (i + 1) * data_slicer]
            centers = self.subvector_estimators[cluster_index][i].cluster_centers_  
            distances_table[:, i, :] = euclidean_distances(query, centers, squared=True)
        #! calculate the distance between the query vectors and the codewords
        distances = np.zeros((queries.shape[0], len(codewords)), dtype=np.float32)
        for i in range(self.n_subvectors):
            distances += distances_table[:, i, codewords[:, i]]
        return distances

    def _searchPQ(self, query_vectors, codewords,cluster_index,n_neighbors=1):
        if not self.is_trained:
            raise ValueError("Index is not trained yet")
        #! calculate the distance between the query vector and the codewords
        distances = self._subvector_distance(query_vectors, codewords,cluster_index)
        #! get the nearest vectors indices
        nearest_vector_indices = np.argsort(distances, axis=1)[:, :n_neighbors]
        return nearest_vector_indices
    def retreive(self,query_vector,cosine_similarity,get_row,n_clusters=3,n_neighbors=10):
        #! calculate the similarities between the query vector and the cluster centers
        similarities = np.array([cosine_similarity(query_vector, center) for center in self.cluster_centers])
        #! get the n nearest clusters
        nearest_clusters = np.argpartition(similarities, -n_clusters)[-n_clusters:]
        #! get nearest n vectors
        similarities = np.empty((0,))
        vectors = np.empty((0, self.dimension))
        for cluster in nearest_clusters:
            #! get the codewords and indices of the vectors in the cluster
            codewords, indices = self.inverted_index[cluster]
            nearest_vector_indices=self._searchPQ(query_vectors=query_vector.reshape(1,-1),codewords=codewords,n_neighbors=n_neighbors*2,cluster_index=cluster).flatten()
            new_vectors = np.array([get_row(i) for i in indices[nearest_vector_indices]])
            new_similarities = np.array([cosine_similarity(query_vector, vector) for vector in new_vectors])
            vectors = np.append(vectors, new_vectors,axis=0)
            similarities = np.append(similarities, new_similarities) 
            
        nearest_arrays = np.argpartition(similarities, -n_neighbors)[-n_neighbors:]
        return vectors[nearest_arrays]
    def save_index(self,file_name):
        with open(file_name, 'wb') as f:
            np.save(f, self.cluster_centers)
            pickle.dump(self.inverted_index, f)
            pickle.dump(self.subvector_estimators, f)
    def load_index(self, file_name):
        with open(file_name, 'rb') as f:
            self.cluster_centers = np.load(f)
            self.inverted_index = pickle.load(f)
            self.subvector_estimators = pickle.load(f)
        self.n_clusters=self.cluster_centers.shape[0]
        self.n_subvectors=len(self.subvector_estimators[0])
        self.n_clusters_per_subvector=len(self.subvector_estimators[0][0].cluster_centers_)
        self.dimension=self.cluster_centers.shape[1]
        self.n_bits=int(np.log2(self.n_clusters_per_subvector))
        self.is_trained = True


In [32]:
VecDB=vec_db.VecDB(db_size=10000000)
random_vectors = VecDB.get_all_rows()
index=IVF_PQ_Index(n_subvectors=14,n_bits=8,n_clusters=30)
index.fit(random_vectors)
index.save_index(VecDB.index_path)

INFO:root:Created subvector estimators [KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state=42), KMeans(init='random', max_iter=50, n_clusters=256, random_state

Initialization complete
Iteration 0, inertia 83243264.0.
Iteration 1, inertia 54807908.0.
Iteration 2, inertia 54751932.0.
Iteration 3, inertia 54735752.0.
Iteration 4, inertia 54726784.0.
Iteration 5, inertia 54720920.0.
Iteration 6, inertia 54716676.0.
Iteration 7, inertia 54713772.0.
Iteration 8, inertia 54711656.0.
Iteration 9, inertia 54710016.0.
Iteration 10, inertia 54708624.0.
Iteration 11, inertia 54707712.0.
Iteration 12, inertia 54706896.0.
Iteration 13, inertia 54706164.0.
Iteration 14, inertia 54705584.0.
Iteration 15, inertia 54704800.0.
Iteration 16, inertia 54704344.0.
Iteration 17, inertia 54703792.0.
Iteration 18, inertia 54703256.0.
Iteration 19, inertia 54702732.0.
Iteration 20, inertia 54702196.0.
Iteration 21, inertia 54701664.0.
Iteration 22, inertia 54701140.0.
Iteration 23, inertia 54700716.0.
Iteration 24, inertia 54700308.0.
Iteration 25, inertia 54700080.0.
Iteration 26, inertia 54699792.0.
Iteration 27, inertia 54699384.0.
Iteration 28, inertia 54698968.0.


INFO:root:Fitting KMeans for the 1-th subvectors
INFO:root:Fitting KMeans for the 2-th subvectors
INFO:root:Fitting KMeans for the 3-th subvectors
INFO:root:Fitting KMeans for the 4-th subvectors
INFO:root:Fitting KMeans for the 5-th subvectors
INFO:root:Fitting KMeans for the 6-th subvectors
INFO:root:Fitting KMeans for the 7-th subvectors
INFO:root:Fitting KMeans for the 8-th subvectors
INFO:root:Fitting KMeans for the 9-th subvectors
INFO:root:Fitting KMeans for the 10-th subvectors
INFO:root:Fitting KMeans for the 11-th subvectors
INFO:root:Fitting KMeans for the 12-th subvectors
INFO:root:Fitting KMeans for the 13-th subvectors
INFO:root:Fitting KMeans for the 14-th subvectors
INFO:root:Fitting KMeans for the 1-th subvectors
INFO:root:Fitting KMeans for the 2-th subvectors
INFO:root:Fitting KMeans for the 3-th subvectors
INFO:root:Fitting KMeans for the 4-th subvectors
INFO:root:Fitting KMeans for the 5-th subvectors
INFO:root:Fitting KMeans for the 6-th subvectors
INFO:root:Fitti

In [33]:
index.load_index(VecDB.index_path)

In [34]:
query_vectors = np.random.rand(10, 70)
nearest_vectors = []
for query_vector in query_vectors:
    similarities = np.array([VecDB._cal_score(query_vector, vector) for vector in random_vectors])
    nearest_vector_indices = np.argpartition(similarities, -10)[-10:]
    nearest_vectors.append(random_vectors[nearest_vector_indices])
nearest_vectors = np.array(nearest_vectors)
# print(nearest_vectors)

In [35]:
results=[]
for query_vector in query_vectors:
    results.append(index.retreive(query_vector,VecDB._cal_score,VecDB.get_one_row,n_clusters=15,n_neighbors=10))
results=np.array(results)
# print(results)

In [36]:
print(nearest_vectors.shape)

(10, 10, 70)


In [37]:
for i in range(query_vectors.shape[0]):
    print(f"Query {i+1}")
    for j in range(10):
        print(np.all(nearest_vectors[i][j]==results[i][j]))
   

Query 1
False
False
False
True
False
True
False
False
False
False
Query 2
False
False
False
False
False
False
False
False
False
False
Query 3
False
True
False
False
False
True
False
False
False
False
Query 4
False
False
False
False
False
False
False
False
False
False
Query 5
False
False
False
False
False
False
False
False
True
False
Query 6
False
False
False
False
False
False
False
False
False
False
Query 7
False
False
True
False
False
False
False
False
False
False
Query 8
False
False
False
False
False
False
False
False
False
False
Query 9
False
False
False
False
False
True
True
False
True
False
Query 10
False
False
False
False
False
False
False
False
False
False
